In [126]:
import os
import psycopg2
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

from config import DB_VISION_ZERO, DB_MOPED

In [127]:
def get_data(query, cursor):
    """
    Get data from database
    """
    cursor.execute(query)
    data = cursor.fetchall()
    field_names = [i[0] for i in cursor.description]
    df = pd.DataFrame(data, columns=field_names)

    return df

conn_vz = psycopg2.connect(
    dbname = DB_VISION_ZERO['dbname'],
    user = DB_VISION_ZERO["user"],
    host = DB_VISION_ZERO["host"],
    password = DB_VISION_ZERO["password"],
    port=5432
)

conn_moped = psycopg2.connect(
    dbname = DB_MOPED["dbname"],
    user = DB_MOPED["user"],
    host = DB_MOPED["host"],
    password = DB_MOPED["password"],
    port = 5432
)

cursor_vz = conn_vz.cursor()
cursor_moped = conn_moped.cursor()

# Moped processing

In [128]:
# Creating moped dataframe
QUERY_MOPED = """SELECT project_id, project_component_id, geometry, 
line_geometry, substantial_completion_date,
component_name, component_name_full, component_subtype, 
component_work_types, type_name FROM component_arcgis_online_view"""

# Creating moped dataframe
df_moped = get_data(QUERY_MOPED, cursor_moped)

In [129]:
# Data frame info
df_moped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12228 entries, 0 to 12227
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   project_id                   12228 non-null  int64              
 1   project_component_id         11783 non-null  float64            
 2   geometry                     11783 non-null  object             
 3   line_geometry                11783 non-null  object             
 4   substantial_completion_date  2916 non-null   datetime64[ns, UTC]
 5   component_name               12228 non-null  object             
 6   component_name_full          12228 non-null  object             
 7   component_subtype            8861 non-null   object             
 8   component_work_types         2257 non-null   object             
 9   type_name                    419 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1

In [130]:
# Dropping observations where substantial completion date or line geometry is absent
df_moped_filter = df_moped.dropna(subset=['substantial_completion_date', 'line_geometry'])
df_moped_filter.head()

,project_id,project_component_id,geometry,line_geometry,substantial_completion_date,component_name,component_name_full,component_subtype,component_work_types,type_name
10,12,181.0,"{'type': 'MultiPoint', 'coordinates': [[-97.73...","{'type': 'LineString', 'coordinates': [[-97.73...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod
11,12,183.0,"{'type': 'MultiPoint', 'coordinates': [[-97.73...","{'type': 'LineString', 'coordinates': [[-97.73...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod
12,12,182.0,"{'type': 'MultiPoint', 'coordinates': [[-97.73...","{'type': 'LineString', 'coordinates': [[-97.73...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod
13,12,469.0,"{'type': 'MultiPoint', 'coordinates': [[-97.73...","{'type': 'MultiLineString', 'coordinates': [[[...",2022-10-10 05:00:00+00:00,Intersection,Intersection - Improvement,Improvement,None,Signal - Mod
14,12,16.0,"{'type': 'MultiPoint', 'coordinates': [[-97.73...","{'type': 'LineString', 'coordinates': [[-97.73...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod


In [131]:
df_moped_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2733 entries, 10 to 12227
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   project_id                   2733 non-null   int64              
 1   project_component_id         2733 non-null   float64            
 2   geometry                     2733 non-null   object             
 3   line_geometry                2733 non-null   object             
 4   substantial_completion_date  2733 non-null   datetime64[ns, UTC]
 5   component_name               2733 non-null   object             
 6   component_name_full          2733 non-null   object             
 7   component_subtype            2189 non-null   object             
 8   component_work_types         835 non-null    object             
 9   type_name                    84 non-null     object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), ob

In [132]:
# Convert timestamp columns to string
timestamp_columns = ["substantial_completion_date"]

for col in timestamp_columns:
    df_moped_filter.loc[:, col] = df_moped_filter[col].astype(str)

# Apply the geometry transformation
df_moped_filter.loc[:, "geometry"] = df_moped_filter["geometry"].apply(lambda x: shape(x) if x is not None else None)
df_moped_filter.loc[:, "line_geometry"] = df_moped_filter["line_geometry"].apply(lambda x: shape(x) if x is not None else None)

# Create GeoDataFrame
gdf_moped = gpd.GeoDataFrame(df_moped_filter, geometry="geometry")

In [133]:
# Adding a unique ID column
gdf_moped.insert(0, 'moped_component_id', range(1, 1 + len(gdf_moped)))

In [134]:
gdf_moped.head()

,moped_component_id,project_id,project_component_id,geometry,line_geometry,substantial_completion_date,component_name,component_name_full,component_subtype,component_work_types,type_name
10,1,12,181.0,MULTIPOINT (-97.73351 30.26751),"LINESTRING (-97.733436244 30.267508296, -97.73...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod
11,2,12,183.0,MULTIPOINT (-97.73430 30.26772),"LINESTRING (-97.73422624299999 30.267724297, -...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod
12,3,12,182.0,MULTIPOINT (-97.73386 30.26657),"LINESTRING (-97.73378324399999 30.266570296, -...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod
13,4,12,469.0,"MULTIPOINT (-97.73467 30.26683, -97.73431 30.2...","MULTILINESTRING ((-97.733437244 30.267507296, ...",2022-10-10 05:00:00+00:00,Intersection,Intersection - Improvement,Improvement,None,Signal - Mod
14,5,12,16.0,MULTIPOINT (-97.73467 30.26682),"LINESTRING (-97.734592244 30.266820297, -97.73...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod


# VisionZero processing

In [135]:
# Creaing vision zero dataframe
QUERY_CRASH_DATA = """SELECT crash_id, crash_fatal_fl, crash_date,
road_constr_zone_fl, latitude, longitude, tot_injry_cnt, 
death_cnt, est_comp_cost FROM atd_txdot_crashes"""

df_vz = get_data(QUERY_CRASH_DATA, cursor_vz)

In [136]:
df_vz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419527 entries, 0 to 419526
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   crash_id             419527 non-null  int64  
 1   crash_fatal_fl       419527 non-null  object 
 2   crash_date           419527 non-null  object 
 3   road_constr_zone_fl  419525 non-null  object 
 4   latitude             387958 non-null  float64
 5   longitude            387958 non-null  float64
 6   tot_injry_cnt        419525 non-null  float64
 7   death_cnt            419526 non-null  float64
 8   est_comp_cost        419525 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 28.8+ MB


In [137]:
# Keepiing only those observations where x-y coordinates are present
df_vz_filter = df_vz[df_vz['latitude'].notnull() & df_vz['longitude'].notnull()]

In [138]:
df_vz_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387958 entries, 0 to 419526
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   crash_id             387958 non-null  int64  
 1   crash_fatal_fl       387958 non-null  object 
 2   crash_date           387958 non-null  object 
 3   road_constr_zone_fl  387958 non-null  object 
 4   latitude             387958 non-null  float64
 5   longitude            387958 non-null  float64
 6   tot_injry_cnt        387958 non-null  float64
 7   death_cnt            387958 non-null  float64
 8   est_comp_cost        387958 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 29.6+ MB


In [139]:
# Convert timestamp columns to string
timestamp_columns = ["crash_date"]

for col in timestamp_columns:
    df_vz_filter.loc[:, col] = df_vz_filter[col].astype(str)

In [140]:
# Creating geodataframe
gdf_vz = gpd.GeoDataFrame(df_vz_filter,
                          geometry=gpd.points_from_xy(df_vz_filter.longitude,
                                                      df_vz_filter.latitude),
                                                      crs='EPSG:4326')

gdf_vz.head()

,crash_id,crash_fatal_fl,crash_date,road_constr_zone_fl,latitude,longitude,tot_injry_cnt,death_cnt,est_comp_cost,geometry
0,15070581,N,2016-04-26,N,30.244782,-97.583278,0.0,0.0,255000.00,POINT (-97.58328 30.24478)
1,14452761,N,2015-05-13,N,30.325035,-97.743902,1.0,0.0,284000.00,POINT (-97.74390 30.32503)
2,18461021,N,2021-09-06,N,30.228375,-97.789327,1.0,0.0,386000.00,POINT (-97.78933 30.22838)
3,18461024,N,2021-09-06,N,30.160195,-97.736942,0.0,0.0,102000.00,POINT (-97.73694 30.16019)
4,19292910,N,2022-12-15,N,30.649952,-97.679346,0.0,0.0,102000.00,POINT (-97.67935 30.64995)


# Spatial join

In [141]:
# Creating buffer for joining
gdf_moped = gdf_moped.set_geometry('line_geometry')
gdf_moped.set_crs(epsg=4326, inplace=True)
gdf_moped_proj = gdf_moped.to_crs(epsg=32614)
buffer_distance = 20

gdf_moped_proj = gdf_moped.to_crs(epsg=32614)

In [142]:
gdf_moped_proj['buffered_geometry'] = gdf_moped_proj.geometry.buffer(buffer_distance)
buffered_moped_gdf = gdf_moped_proj.set_geometry('buffered_geometry').to_crs('EPSG:4326')

Buffered geometry results in line strings and multi line strings being turned into polygons

In [143]:
buffered_moped_gdf.head()

,moped_component_id,project_id,project_component_id,geometry,line_geometry,substantial_completion_date,component_name,component_name_full,component_subtype,component_work_types,type_name,buffered_geometry
10,1,12,181.0,MULTIPOINT (-97.73351 30.26751),"LINESTRING (621833.101 3349106.852, 621832.966...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod,"POLYGON ((-97.73323 30.26752, -97.73323 30.267..."
11,2,12,183.0,MULTIPOINT (-97.73430 30.26772),"LINESTRING (621756.837 3349129.944, 621756.702...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod,"POLYGON ((-97.73402 30.26774, -97.73402 30.267..."
12,3,12,182.0,MULTIPOINT (-97.73386 30.26657),"LINESTRING (621800.878 3349002.525, 621800.743...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod,"POLYGON ((-97.73358 30.26659, -97.73358 30.266..."
13,4,12,469.0,"MULTIPOINT (-97.73467 30.26683, -97.73431 30.2...","MULTILINESTRING ((621833.006 3349106.740, 6218...",2022-10-10 05:00:00+00:00,Intersection,Intersection - Improvement,Improvement,None,Signal - Mod,"MULTIPOLYGON (((-97.73323 30.26752, -97.73323 ..."
14,5,12,16.0,MULTIPOINT (-97.73467 30.26682),"LINESTRING (621722.743 3349029.365, 621722.609...",2022-10-10 05:00:00+00:00,Signal,Signal - Traffic,Traffic,Modification,Signal - Mod,"POLYGON ((-97.73439 30.26684, -97.73438 30.266..."


In [144]:
# Spatial join
crashes_near_projects = gpd.sjoin(gdf_vz, buffered_moped_gdf, how='inner')

# Creating a unique ID column
crashes_near_projects['crash_project_component_id'] = crashes_near_projects['crash_id'].astype(str) + "-" + crashes_near_projects['project_id'].astype(str) + "-" + crashes_near_projects['project_component_id'].astype(str)

In [145]:
print('Number of unique crashes in merged dataset:', crashes_near_projects['crash_id'].nunique())
print('Number of unique moped component IDs in merged dataset:', crashes_near_projects['moped_component_id'].nunique())

Number of unique crashes in merged dataset: 99907
Number of unique moped component IDs in merged dataset: 2067


In [146]:
crashes_near_projects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193164 entries, 1 to 418585
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   crash_id                     193164 non-null  int64              
 1   crash_fatal_fl               193164 non-null  object             
 2   crash_date                   193164 non-null  object             
 3   road_constr_zone_fl          193164 non-null  object             
 4   latitude                     193164 non-null  float64            
 5   longitude                    193164 non-null  float64            
 6   tot_injry_cnt                193164 non-null  float64            
 7   death_cnt                    193164 non-null  float64            
 8   est_comp_cost                193164 non-null  object             
 9   geometry_left                193164 non-null  geometry           
 10  index_right                  193164 n

# Analysis

In [147]:
# Formatting crash date
crashes_near_projects['crash_date'] = pd.to_datetime(crashes_near_projects['crash_date'], errors='coerce').dt.tz_localize('UTC', nonexistent='NaT', ambiguous='NaT').dt.tz_convert('UTC')

In [148]:
crashes_near_projects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193164 entries, 1 to 418585
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   crash_id                     193164 non-null  int64              
 1   crash_fatal_fl               193164 non-null  object             
 2   crash_date                   193164 non-null  datetime64[ns, UTC]
 3   road_constr_zone_fl          193164 non-null  object             
 4   latitude                     193164 non-null  float64            
 5   longitude                    193164 non-null  float64            
 6   tot_injry_cnt                193164 non-null  float64            
 7   death_cnt                    193164 non-null  float64            
 8   est_comp_cost                193164 non-null  object             
 9   geometry_left                193164 non-null  geometry           
 10  index_right                  193164 n

In [149]:
# Re-arranging columns
# unique identifier for each observation
crashes_near_projects.insert(0, 'crash_project_component_id', crashes_near_projects.pop('crash_project_component_id'))

# moped_component_id
crashes_near_projects.insert(2, 'moped_component_id', crashes_near_projects.pop('moped_component_id'))

# crash_date
crashes_near_projects.insert(4, 'crash_date', crashes_near_projects.pop('crash_date'))

# project compoenent ID
crashes_near_projects.insert(3, 'project_component_id', crashes_near_projects.pop('project_component_id'))

# Substantial completion date
crashes_near_projects.insert(5, 'substantial_completion_date', crashes_near_projects.pop('substantial_completion_date'))

In [150]:
# Creating a binary version of the fatality column
crashes_near_projects['crash_fatal_binary'] = crashes_near_projects['crash_fatal_fl'].apply(lambda x: 1 if x == "Y" else 0)
crashes_near_projects.pop('crash_fatal_fl')

# Rearranging the crash fatal binary column 
crashes_near_projects.insert(4, 'crash_fatal_binary', crashes_near_projects.pop('crash_fatal_binary'))

In [151]:
crashes_near_projects.head()

,crash_project_component_id,crash_id,moped_component_id,project_component_id,crash_fatal_binary,substantial_completion_date,crash_date,road_constr_zone_fl,latitude,longitude,...,geometry_left,index_right,project_id,geometry_right,line_geometry,component_name,component_name_full,component_subtype,component_work_types,type_name
1,14452761-832-1113.0,14452761,355,1113.0,0,2010-04-19 05:00:00+00:00,2015-05-13 00:00:00+00:00,N,30.325035,-97.743902,...,POINT (-97.74390 30.32503),1166,832,"MULTILINESTRING ((-97.74915 30.32764, -97.7478...","MULTILINESTRING ((620247.917 3355754.490, 6203...",Bike Lane,Bike Lane,None,Lane Conversion,None
4496,16094106-832-1113.0,16094106,355,1113.0,0,2010-04-19 05:00:00+00:00,2017-11-20 00:00:00+00:00,N,30.327692,-97.749271,...,POINT (-97.74927 30.32769),1166,832,"MULTILINESTRING ((-97.74915 30.32764, -97.7478...","MULTILINESTRING ((620247.917 3355754.490, 6203...",Bike Lane,Bike Lane,None,Lane Conversion,None
5974,17565790-832-1113.0,17565790,355,1113.0,0,2010-04-19 05:00:00+00:00,2020-02-12 00:00:00+00:00,N,30.325825,-97.739452,...,POINT (-97.73945 30.32582),1166,832,"MULTILINESTRING ((-97.74915 30.32764, -97.7478...","MULTILINESTRING ((620247.917 3355754.490, 6203...",Bike Lane,Bike Lane,None,Lane Conversion,None
9255,18654296-832-1113.0,18654296,355,1113.0,0,2010-04-19 05:00:00+00:00,2021-12-17 00:00:00+00:00,N,30.321099,-97.729392,...,POINT (-97.72939 30.32110),1166,832,"MULTILINESTRING ((-97.74915 30.32764, -97.7478...","MULTILINESTRING ((620247.917 3355754.490, 6203...",Bike Lane,Bike Lane,None,Lane Conversion,None
9535,15053992-832-1113.0,15053992,355,1113.0,0,2010-04-19 05:00:00+00:00,2016-04-26 00:00:00+00:00,N,30.321095,-97.729382,...,POINT (-97.72938 30.32109),1166,832,"MULTILINESTRING ((-97.74915 30.32764, -97.7478...","MULTILINESTRING ((620247.917 3355754.490, 6203...",Bike Lane,Bike Lane,None,Lane Conversion,None


In [152]:
crashes_near_projects['crash_fatal_binary'].value_counts()

crash_fatal_binary
0    192356
1       808
Name: count, dtype: int64

In [153]:
crashes_near_projects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193164 entries, 1 to 418585
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   crash_project_component_id   193164 non-null  object             
 1   crash_id                     193164 non-null  int64              
 2   moped_component_id           193164 non-null  int64              
 3   project_component_id         193164 non-null  float64            
 4   crash_fatal_binary           193164 non-null  int64              
 5   substantial_completion_date  193164 non-null  datetime64[ns, UTC]
 6   crash_date                   193164 non-null  datetime64[ns, UTC]
 7   road_constr_zone_fl          193164 non-null  object             
 8   latitude                     193164 non-null  float64            
 9   longitude                    193164 non-null  float64            
 10  tot_injry_cnt                193164 n

In [154]:
# Creating indicator variables for crash occuring pre and post completion of mobility project
crashes_near_projects.insert(7, 'crash_pre_completion', crashes_near_projects['crash_date'] < crashes_near_projects['substantial_completion_date'])
crashes_near_projects.insert(8, 'crash_post_completion', crashes_near_projects['crash_date'] > crashes_near_projects['substantial_completion_date'])

In [155]:
# Creating time difference variables
crashes_near_projects.insert(9, 'crash_project_date_diff', crashes_near_projects['substantial_completion_date'] - crashes_near_projects['crash_date'])

In [156]:
# Converting estimated comp cost to float format
crashes_near_projects['est_comp_cost'] = crashes_near_projects['est_comp_cost'].map(lambda x: float(x))

crashes_near_projects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193164 entries, 1 to 418585
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   crash_project_component_id   193164 non-null  object             
 1   crash_id                     193164 non-null  int64              
 2   moped_component_id           193164 non-null  int64              
 3   project_component_id         193164 non-null  float64            
 4   crash_fatal_binary           193164 non-null  int64              
 5   substantial_completion_date  193164 non-null  datetime64[ns, UTC]
 6   crash_date                   193164 non-null  datetime64[ns, UTC]
 7   crash_pre_completion         193164 non-null  bool               
 8   crash_post_completion        193164 non-null  bool               
 9   crash_project_date_diff      193164 non-null  timedelta64[ns]    
 10  road_constr_zone_fl          193164 n

In [157]:
# Function to calculate duration in years
def calculate_duration(df, date_col1, date_col2):
    duration = (df[date_col2] - df[date_col1]).dt.total_seconds() / (365.25 * 24 * 3600)
    return duration

crashes_near_projects['pre_completion_duration'] = crashes_near_projects['crash_pre_completion'] * calculate_duration(crashes_near_projects, 'crash_date', 'substantial_completion_date')
crashes_near_projects['post_completion_duration'] = crashes_near_projects['crash_post_completion'] * calculate_duration(crashes_near_projects, 'substantial_completion_date', 'crash_date')

pre_completion_stats = crashes_near_projects[crashes_near_projects['crash_pre_completion'] == True].groupby('moped_component_id').agg({
    'crash_id': 'count',
    'pre_completion_duration': 'sum',
    'crash_fatal_binary': 'sum',
    'tot_injry_cnt': 'sum',
    'death_cnt': 'sum',
    'est_comp_cost': 'sum'
}).rename(columns={'crash_id': 'pre_crash_count',
                   'crash_fatal_binary': 'pre_fatal_crash_count',
                   'tot_injry_cnt': 'pre_total_injury_count',
                   'death_cnt': 'pre_total_death_count',
                   'est_comp_cost': 'pre_est_comp_cost'}).reset_index()

post_completion_stats = crashes_near_projects[crashes_near_projects['crash_post_completion'] == True].groupby('moped_component_id').agg({
    'crash_id': 'count',
    'post_completion_duration': 'sum',
    'crash_fatal_binary': 'sum',
    'tot_injry_cnt': 'sum',
    'death_cnt': 'sum',
    'est_comp_cost': 'sum'
}).rename(columns={'crash_id': 'post_crash_count',
                   'crash_fatal_binary': 'post_fatal_crash_count',
                   'tot_injry_cnt': 'post_total_injury_count',
                   'death_cnt': 'post_total_death_count',
                   'est_comp_cost': 'post_est_comp_cost'}).reset_index()


# Merging
annualized_statistics = pre_completion_stats.merge(post_completion_stats, on='moped_component_id', how='outer').fillna(0)

# Calculating annualized statistics
# Crash rate
annualized_statistics['pre_annualized_crash_rate'] = annualized_statistics['pre_crash_count'] / annualized_statistics['pre_completion_duration']
annualized_statistics['post_annualized_crash_rate'] = annualized_statistics['post_crash_count'] / annualized_statistics['post_completion_duration']

# Fatality
annualized_statistics['pre_annualized_fatal_crash_rate'] = annualized_statistics['pre_fatal_crash_count'] / annualized_statistics['pre_completion_duration']
annualized_statistics['post_annualized_fatal_crash_rate'] = annualized_statistics['post_fatal_crash_count'] / annualized_statistics['post_completion_duration']

# Injury count
annualized_statistics['pre_annualized_injury_rate'] = annualized_statistics['pre_total_injury_count'] / annualized_statistics['pre_completion_duration']
annualized_statistics['post_annualized_injury_rate'] = annualized_statistics['post_total_injury_count'] / annualized_statistics['post_completion_duration']

# Death count
annualized_statistics['pre_annualized_death_rate'] = annualized_statistics['pre_total_death_count'] / annualized_statistics['pre_completion_duration']
annualized_statistics['post_annualized_death_rate'] = annualized_statistics['post_total_death_count'] / annualized_statistics['post_completion_duration']

# Estimated cost
annualized_statistics['pre_annualized_cost'] = annualized_statistics['pre_est_comp_cost'] / annualized_statistics['pre_completion_duration']
annualized_statistics['post_annualized_cost'] = annualized_statistics['post_est_comp_cost'] / annualized_statistics['post_completion_duration']

In [158]:
# Getting completion date for each moped component id
completion_dates = crashes_near_projects.groupby('moped_component_id')['substantial_completion_date'].first().reset_index()

# Merging into the annualized crash rate DataFrame
annualized_statistics = annualized_statistics.merge(completion_dates, on='moped_component_id', how='left')

In [159]:
annualized_statistics = annualized_statistics[['moped_component_id',
                                               'substantial_completion_date', 
                                               'pre_annualized_crash_rate', 
                                               'post_annualized_crash_rate',
                                               'pre_annualized_fatal_crash_rate',
                                               'post_annualized_fatal_crash_rate',
                                               'pre_annualized_injury_rate',
                                               'post_annualized_injury_rate',
                                               'pre_annualized_death_rate',
                                               'post_annualized_death_rate',
                                               'pre_annualized_cost',
                                               'post_annualized_cost'
                                               ]]

In [160]:
annualized_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067 entries, 0 to 2066
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   moped_component_id                2067 non-null   int64              
 1   substantial_completion_date       2067 non-null   datetime64[ns, UTC]
 2   pre_annualized_crash_rate         1867 non-null   float64            
 3   post_annualized_crash_rate        1452 non-null   float64            
 4   pre_annualized_fatal_crash_rate   1867 non-null   float64            
 5   post_annualized_fatal_crash_rate  1452 non-null   float64            
 6   pre_annualized_injury_rate        1867 non-null   float64            
 7   post_annualized_injury_rate       1452 non-null   float64            
 8   pre_annualized_death_rate         1867 non-null   float64            
 9   post_annualized_death_rate        1452 non-null   float64      

In [161]:
# Creating difference columns between pre and post
annualized_statistics.insert(4, 'delta_crash_rate', annualized_statistics['post_annualized_crash_rate']  - annualized_statistics['pre_annualized_crash_rate'])
annualized_statistics.insert(7, 'delta_fatal_crash_rate', annualized_statistics['post_annualized_fatal_crash_rate']  - annualized_statistics['pre_annualized_fatal_crash_rate'])
annualized_statistics.insert(10, 'delta_injury_rate', annualized_statistics['post_annualized_injury_rate']  - annualized_statistics['pre_annualized_injury_rate'])
annualized_statistics.insert(13, 'delta_death_rate', annualized_statistics['post_annualized_death_rate']  - annualized_statistics['pre_annualized_death_rate'])
annualized_statistics.insert(16, 'delta_comp_cost', annualized_statistics['post_annualized_cost']  - annualized_statistics['pre_annualized_cost'])


In [162]:
# Merging additional information such as component name, type, etc.
additional_info = crashes_near_projects[['moped_component_id', 
                                         'component_name', 
                                         'component_name_full', 
                                         'component_subtype',
                                         'component_work_types', 
                                         'type_name',
                                         'line_geometry']].drop_duplicates()

annualized_statistics = annualized_statistics.merge(additional_info, on='moped_component_id', how='left')

In [163]:
annualized_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067 entries, 0 to 2066
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   moped_component_id                2067 non-null   int64              
 1   substantial_completion_date       2067 non-null   datetime64[ns, UTC]
 2   pre_annualized_crash_rate         1867 non-null   float64            
 3   post_annualized_crash_rate        1452 non-null   float64            
 4   delta_crash_rate                  1252 non-null   float64            
 5   pre_annualized_fatal_crash_rate   1867 non-null   float64            
 6   post_annualized_fatal_crash_rate  1452 non-null   float64            
 7   delta_fatal_crash_rate            1252 non-null   float64            
 8   pre_annualized_injury_rate        1867 non-null   float64            
 9   post_annualized_injury_rate       1452 non-null   float64      

In [164]:
# Reordering
all_columns = annualized_statistics.columns.tolist()

first_column = all_columns[0]
last_six = all_columns[-6:]
new_order = [first_column] + last_six + all_columns[1:-6]
annualized_statistics = annualized_statistics[new_order]

In [165]:
annualized_statistics.to_csv('../Output/annualized_statistics.csv', na_rep="NA", index=False)